In [1]:
from dotenv import load_dotenv
from IPython.lib.display import FileLink
import time
import httpx

load_dotenv()

True

In [2]:
# cd demos/fastapi_server 
# uv run uvicorn main:app --reload

In [3]:
from enum import Enum

BASE_URL = "http://localhost:8000"

class agent_type_enum(Enum):
    agent_no_tools = "agent_no_tools"
    agent_client_tools = "agent_client_tools"
    agent_all_raw = "agent_all_raw"
    agent_all_xml = "agent_all_xml"


def call_agent_api(
    prompt: str | list[dict] | dict,
    agent_uuid: str | None = None,
    agent_type: str | Enum = "agent_all_raw",
    log_file: str | None = None,
) -> str:
    """Call agent API and log streamed response to file.

    Returns:
        Path to the log file
    """
    if isinstance(agent_type, Enum):
        agent_type = agent_type.value

    # Build filename
    timestamp = int(time.time() * 1000)
    if log_file is None:
        log_file = f"temp_logs/{agent_type}_{timestamp}.log"
    else:
        log_file = f"temp_logs/{log_file}_{agent_type}_{timestamp}.log"

    # Build request payload
    payload = {"user_prompt": prompt}
    if agent_uuid:
        payload["agent_uuid"] = agent_uuid
    else:
        payload["agent_type"] = agent_type

    # Stream response and write to file
    with open(log_file, "w") as f:
        with httpx.Client(timeout=None) as client:
            with client.stream("POST", f"{BASE_URL}/agent/run", json=payload) as response:
                for line in response.iter_lines():
                    f.write(line)
                    f.write("\n")
                    # if line.startswith("data: "):
                        # data = line[6:]  # Strip "data: " prefix
                        # if data != "[DONE]":
                            # Unescape newlines and write
                            # f.write(data.replace("\\n", "\n"))
                            # f.write("\n")

    return log_file

In [4]:
path = call_agent_api("Hello", log_file="hello", agent_type=agent_type_enum.agent_all_xml)
FileLink(path)

d:\AI Forge\anthropic-agent\demos\fastapi_server\temp_logs\hello_agent_all_xml_1767460434889.log

# Raw formatted Output

## No tools

In [5]:
path = call_agent_api(
    "Hi, what is AI?", 
    log_file="no_tools_what_is_ai", 
    agent_type=agent_type_enum.agent_no_tools
)
FileLink(path)

/Users/aurosoni/Documents/AI Forge/Repos/anthropic_agent/demos/fastapi_server/temp_logs/no_tools_what_is_ai_agent_no_tools_1767194630354.log

## Client Tools

In [6]:
path = call_agent_api(
    "Hi, what is 5*(3+2/4)?", 
    log_file="client_tools_math_expression", 
    agent_type=agent_type_enum.agent_client_tools
)
FileLink(path)

/Users/aurosoni/Documents/AI Forge/Repos/anthropic_agent/demos/fastapi_server/temp_logs/client_tools_math_expression_agent_client_tools_1767194640147.log

## Server Tools

In [7]:
path = call_agent_api(
    "Hi, what is 5*(3+2/4)?", 
    log_file="server_tools_raw_math_expression", 
    agent_type=agent_type_enum.agent_all_raw
)
FileLink(path)

/Users/aurosoni/Documents/AI Forge/Repos/anthropic_agent/demos/fastapi_server/temp_logs/server_tools_raw_math_expression_agent_all_raw_1767194654379.log

In [8]:
test_image = "../../currency_receipt_usd_jpy.png"
# Convert image to base64
import base64

with open(test_image, "rb") as image_file:
    base64_image = base64.b64encode(image_file.read()).decode('utf-8')

In [9]:
prompt = {
    "role": "user",
    "content": [
        {
            "type": "image",
            "source": {
                "type": "base64",
                "media_type": "image/png",
                "data": base64_image,
            },
        },
        {
            "type": "text",
            "text": "I paid the receipt in USD. What is the amount in INR?"
        },
    ]
}
path = call_agent_api(
    prompt, 
    log_file="image_receipt_usd_to_inr", 
    agent_type=agent_type_enum.agent_all_raw
)
FileLink(path)

/Users/aurosoni/Documents/AI Forge/Repos/anthropic_agent/demos/fastapi_server/temp_logs/image_receipt_usd_to_inr_agent_all_raw_1767194669008.log

In [10]:
prompt = "Fetch me the latest happenings in the Y Combinator ecosystem"
path = call_agent_api(
    prompt,
    log_file="ycombinator_latest_happenings",
    agent_type=agent_type_enum.agent_all_raw
)
FileLink(path)

/Users/aurosoni/Documents/AI Forge/Repos/anthropic_agent/demos/fastapi_server/temp_logs/ycombinator_latest_happenings_agent_all_raw_1767194686201.log

## PDF citations

In [11]:
prompt = [{
    "type": "document",
    "source": {
        "type": "url",
        "url": "https://assets.anthropic.com/m/1cd9d098ac3e6467/original/Claude-3-Model-Card-October-Addendum.pdf"
    },
    "citations": {"enabled": True}
},
{
    "type": "text",
    "text": "What are the key findings in this document?"
}]

path = call_agent_api(
    prompt, 
    log_file="pdf_citations_model_card_key_findings", 
    agent_type=agent_type_enum.agent_all_raw
)
FileLink(path)

/Users/aurosoni/Documents/AI Forge/Repos/anthropic_agent/demos/fastapi_server/temp_logs/pdf_citations_model_card_key_findings_agent_all_raw_1767194719634.log

## Content citations

In [12]:
prompt = {
    "role": "user",
    "content": [
      {
        "type": "document",
        "source": {
          "type": "text",
          "media_type": "text/plain",
          "data": "The grass is green. The sky is blue."
        },
        "title": "My Document",
        "context": "This is a trustworthy document.",
        "citations": {"enabled": True}
        },
        {
        "type": "text",
        "text": "What color is the grass and sky?"
        }
    ]
    }
path = call_agent_api(
    prompt,
    log_file="content_citations_grass_sky",
    agent_type=agent_type_enum.agent_all_raw,
)
FileLink(path)


/Users/aurosoni/Documents/AI Forge/Repos/anthropic_agent/demos/fastapi_server/temp_logs/content_citations_grass_sky_agent_all_raw_1767194743311.log

## Complex markdown Rendering Cases

In [13]:
prompt = "Produce a markdown report with a table + code block + nested bullets about 'How to reduce latency in an API.'"
path = call_agent_api(
    prompt, 
    log_file="markdown_report_latency_in_api", 
    agent_type=agent_type_enum.agent_all_raw
)
FileLink(path)

/Users/aurosoni/Documents/AI Forge/Repos/anthropic_agent/demos/fastapi_server/temp_logs/markdown_report_latency_in_api_agent_all_raw_1767194747369.log

## Image/File Generation

In [14]:
prompt = "Generate a JSON file named `users.json` with 20 synthetic users: id, name, email, country."
path = call_agent_api(
    prompt, 
    log_file="file_generation_users_json", 
    agent_type=agent_type_enum.agent_all_raw
)
FileLink(path)

/Users/aurosoni/Documents/AI Forge/Repos/anthropic_agent/demos/fastapi_server/temp_logs/file_generation_users_json_agent_all_raw_1767194774401.log

## Content Embedded Tags

In [15]:
prompt = """
<chart_config>
**What Is It?**
A **Chart.js JSON config** is a text format that describes how your chart should look and what data it displays. Think of it like a recipe written in a specific format that Chart.js can read. The chart js json config can be used to show charts in the response.

**Basic Structure**:
```json
{
  "type": "bar",           // Chart type: 'bar', 'line', 'pie', 'doughnut', etc.
  "data": {                // Your chart data
    "labels": [...],       // X-axis labels (array of strings)
    "datasets": [...]      // Your data series (array of objects)
  },
  "options": {             // How the chart looks and behaves
    ...
  }
}
```

To show charts, embed Chart.js JSON config between `<chart>` tags.

**Important:** Place the JSON directly inside the tags without escaping or quotes:

```
<chart>
{
  "type": "bar",
  "data": {
    "labels": ["Label 1", "Label 2"],
    "datasets": [{"label": "My Data", "data": [10, 20]}]
  }
}
</chart>
```

**Complete Template**:
```json
{
  "type": "bar",
  "data": {
    "labels": ["Label 1", "Label 2", "Label 3"],
    "datasets": [{
      "label": "My Dataset",
      "data": [10, 20, 30],
      "backgroundColor": "rgba(75, 192, 192, 0.2)",
      "borderColor": "rgba(75, 192, 192, 1)",
      "borderWidth": 1
    }]
  },
  "options": {
    "responsive": true,
    "plugins": {
      "title": {
        "display": true,
        "text": "My Chart Title"
      }
    },
    "scales": {
      "y": {
        "beginAtZero": true
      }
    }
  }
}
```

**Rules**:
✅ **CAN include**: Numbers, Strings, Booleans, Arrays, Objects, Null

❌ **CANNOT include**:
- Functions or code or callbacks (config is serialized)
- Variables (like `myVariable`)
- Comments (remove `//` when saving as JSON)
- Single quotes (use double quotes only)
</chart_config>

<user_prompt>
Present world population demographic data in a chart.
</user_prompt>
"""
path = call_agent_api(
    prompt,
    log_file="content_embedded_tags_world_population_chart",
    agent_type=agent_type_enum.agent_all_raw,
)
FileLink(path)

/Users/aurosoni/Documents/AI Forge/Repos/anthropic_agent/demos/fastapi_server/temp_logs/content_embedded_tags_world_population_chart_agent_all_raw_1767194797358.log

## Literal newlines

In [4]:
prompt = "Explain the behavior of backslash (\\) in a string. Also explain what \\n does. How is it different from \\\\n or \\r\\n?"
print(prompt)

Explain the behavior of backslash (\) in a string. Also explain what \n does. How is it different from \\n or \r\n?


In [5]:
path = call_agent_api(
    prompt,
    log_file="literal_newlines_backslash_behavior",
    agent_type=agent_type_enum.agent_all_raw,
)
FileLink(path)


/Users/aurosoni/Documents/AI Forge/Repos/anthropic_agent/demos/fastapi_server/temp_logs/literal_newlines_backslash_behavior_agent_all_raw_1767196177029.log

## Image Tool Results

In [7]:
# Test with an existing image in your repo
prompt = "Read and analyze the image at ../../currency_receipt_usd_jpy.png"
path = call_agent_api(
    prompt,
    log_file="image_tool_read_file",
    agent_type=agent_type_enum.agent_all_raw,
)
FileLink(path)

/Users/aurosoni/Documents/AI Forge/Repos/anthropic_agent/demos/fastapi_server/temp_logs/image_tool_read_file_agent_all_raw_1767935958427.log

# XML Formatted Output

## Server Tools

In [16]:
path = call_agent_api(
    "Hi, what is 5*(3+2/4)?",
    log_file="xml_server_tools_raw_math_expression",
    agent_type=agent_type_enum.agent_all_xml,
)
FileLink(path)


/Users/aurosoni/Documents/AI Forge/Repos/anthropic_agent/demos/fastapi_server/temp_logs/xml_server_tools_raw_math_expression_agent_all_xml_1767194823039.log

In [17]:
test_image = "../../currency_receipt_usd_jpy.png"
# Convert image to base64
import base64

with open(test_image, "rb") as image_file:
    base64_image = base64.b64encode(image_file.read()).decode('utf-8')

In [18]:
prompt = {
    "role": "user",
    "content": [
        {
            "type": "image",
            "source": {
                "type": "base64",
                "media_type": "image/png",
                "data": base64_image,
            },
        },
        {
            "type": "text",
            "text": "I paid the receipt in USD. What is the amount in INR?"
        },
    ]
}
path = call_agent_api(
    prompt, 
    log_file="image_receipt_usd_to_inr", 
    agent_type=agent_type_enum.agent_all_xml
)
FileLink(path)

/Users/aurosoni/Documents/AI Forge/Repos/anthropic_agent/demos/fastapi_server/temp_logs/image_receipt_usd_to_inr_agent_all_xml_1767194832426.log

In [19]:
prompt = "Fetch me the latest happenings in the Y Combinator ecosystem"
path = call_agent_api(
    prompt,
    log_file="xml_ycombinator_latest_happenings",
    agent_type=agent_type_enum.agent_all_xml
)
FileLink(path)

/Users/aurosoni/Documents/AI Forge/Repos/anthropic_agent/demos/fastapi_server/temp_logs/xml_ycombinator_latest_happenings_agent_all_xml_1767194848099.log

## PDF citations

In [20]:
prompt = [{
    "type": "document",
    "source": {
        "type": "url",
        "url": "https://assets.anthropic.com/m/1cd9d098ac3e6467/original/Claude-3-Model-Card-October-Addendum.pdf",
    },
    "citations": {"enabled": True},
},
{
    "type": "text",
    "text": "What are the key findings in this document?",
}]

path = call_agent_api(
    prompt,
    log_file="xml_pdf_citations_model_card_key_findings",
    agent_type=agent_type_enum.agent_all_xml,
)
FileLink(path)

/Users/aurosoni/Documents/AI Forge/Repos/anthropic_agent/demos/fastapi_server/temp_logs/xml_pdf_citations_model_card_key_findings_agent_all_xml_1767194884668.log

## Content citations

In [21]:
prompt = {
    "role": "user",
    "content": [
        {
            "type": "document",
            "source": {
                "type": "text",
                "media_type": "text/plain",
                "data": "The grass is green. The sky is blue.",
            },
            "title": "My Document",
            "context": "This is a trustworthy document.",
            "citations": {"enabled": True},
        },
        {
            "type": "text",
            "text": "What color is the grass and sky?",
        },
    ],
}

path = call_agent_api(
    prompt,
    log_file="xml_content_citations_grass_sky",
    agent_type=agent_type_enum.agent_all_xml,
)
FileLink(path)


/Users/aurosoni/Documents/AI Forge/Repos/anthropic_agent/demos/fastapi_server/temp_logs/xml_content_citations_grass_sky_agent_all_xml_1767194905386.log

## Complex markdown Rendering Cases

In [22]:
prompt = "Produce a markdown report with a table + code block + nested bullets about 'How to reduce latency in an API.'"
path = call_agent_api(
    prompt,
    log_file="xml_markdown_report_latency_in_api",
    agent_type=agent_type_enum.agent_all_xml,
)
FileLink(path)

/Users/aurosoni/Documents/AI Forge/Repos/anthropic_agent/demos/fastapi_server/temp_logs/xml_markdown_report_latency_in_api_agent_all_xml_1767194909708.log

## Image/File Generation

In [23]:
prompt = "Generate a JSON file named `users.json` with 20 synthetic users: id, name, email, country."
path = call_agent_api(
    prompt,
    log_file="xml_file_generation_users_json",
    agent_type=agent_type_enum.agent_all_xml,
)
FileLink(path)

/Users/aurosoni/Documents/AI Forge/Repos/anthropic_agent/demos/fastapi_server/temp_logs/xml_file_generation_users_json_agent_all_xml_1767194962115.log

## Content Embedded Tags

In [24]:
prompt = """
<chart_config>
**What Is It?**
A **Chart.js JSON config** is a text format that describes how your chart should look and what data it displays. Think of it like a recipe written in a specific format that Chart.js can read. The chart js json config can be used to show charts in the response.

**Basic Structure**:
```json
{
  "type": "bar",           // Chart type: 'bar', 'line', 'pie', 'doughnut', etc.
  "data": {                // Your chart data
    "labels": [...],       // X-axis labels (array of strings)
    "datasets": [...]      // Your data series (array of objects)
  },
  "options": {             // How the chart looks and behaves
    ...
  }
}
```

To show charts, embed Chart.js JSON config between `<chart>` tags.

**Important:** Place the JSON directly inside the tags without escaping or quotes:

```
<chart>
{
  "type": "bar",
  "data": {
    "labels": ["Label 1", "Label 2"],
    "datasets": [{"label": "My Data", "data": [10, 20]}]
  }
}
</chart>
```

**Complete Template**:
```json
{
  "type": "bar",
  "data": {
    "labels": ["Label 1", "Label 2", "Label 3"],
    "datasets": [{
      "label": "My Dataset",
      "data": [10, 20, 30],
      "backgroundColor": "rgba(75, 192, 192, 0.2)",
      "borderColor": "rgba(75, 192, 192, 1)",
      "borderWidth": 1
    }]
  },
  "options": {
    "responsive": true,
    "plugins": {
      "title": {
        "display": true,
        "text": "My Chart Title"
      }
    },
    "scales": {
      "y": {
        "beginAtZero": true
      }
    }
  }
}
```

**Rules**:
✅ **CAN include**: Numbers, Strings, Booleans, Arrays, Objects, Null

❌ **CANNOT include**:
- Functions or code or callbacks (config is serialized)
- Variables (like `myVariable`)
- Comments (remove `//` when saving as JSON)
- Single quotes (use double quotes only)
</chart_config>

<user_prompt>
Present world population demographic data in a chart.
</user_prompt>
"""
path = call_agent_api(
    prompt,
    log_file="xml_content_embedded_tags_world_population_chart",
    agent_type=agent_type_enum.agent_all_xml,
)
FileLink(path)

/Users/aurosoni/Documents/AI Forge/Repos/anthropic_agent/demos/fastapi_server/temp_logs/xml_content_embedded_tags_world_population_chart_agent_all_xml_1767194993524.log

## Literal Newlines

In [6]:
prompt = "Explain the behavior of backslash (\\) in a string. Also explain what \\n does. How is it different from \\\\n or \\r\\n?"
print(prompt)

Explain the behavior of backslash (\) in a string. Also explain what \n does. How is it different from \\n or \r\n?


In [7]:
path = call_agent_api(
    prompt,
    log_file="xml_literal_newlines_backslash_behavior",
    agent_type=agent_type_enum.agent_all_xml,
)
FileLink(path)


/Users/aurosoni/Documents/AI Forge/Repos/anthropic_agent/demos/fastapi_server/temp_logs/xml_literal_newlines_backslash_behavior_agent_all_xml_1767196197302.log

## Image Tool Results

In [8]:
# Test with an existing image in your repo
prompt = "Read and analyze the image at ../../currency_receipt_usd_jpy.png"
path = call_agent_api(
    prompt,
    log_file="image_tool_read_file",
    agent_type=agent_type_enum.agent_all_xml,
)
FileLink(path)

/Users/aurosoni/Documents/AI Forge/Repos/anthropic_agent/demos/fastapi_server/temp_logs/image_tool_read_file_agent_all_xml_1767936146916.log

# Frontend Tools API Tests

Test the frontend tools flow:
1. Call `/agent/run` with `agent_frontend_tools` type
2. Agent should pause with `awaiting_frontend_tools` when it calls `user_confirm`
3. Parse the pending tool info from the stream
4. Call `/agent/tool_results` to continue execution


In [4]:
# Update the enum to include the new agent type
from enum import Enum
import os

# Ensure temp_logs directory exists
os.makedirs("temp_logs", exist_ok=True)

class agent_type_enum_v2(Enum):
    agent_no_tools = "agent_no_tools"
    agent_client_tools = "agent_client_tools"
    agent_all_raw = "agent_all_raw"
    agent_all_xml = "agent_all_xml"
    agent_frontend_tools = "agent_frontend_tools"  # NEW: includes user_confirm tool


In [5]:
import json
import re
import html

def call_agent_with_frontend_tools(
    prompt: str | list[dict] | dict,
    agent_uuid: str | None = None,
    agent_type: str | Enum = "agent_frontend_tools",
    log_file: str | None = None,
) -> tuple[str, str | None, list[dict] | None]:
    """Call agent API with frontend tools support.
    
    Returns:
        Tuple of (log_file_path, agent_uuid, pending_frontend_tools)
        - pending_frontend_tools is None if agent completed without pausing
    """
    if isinstance(agent_type, Enum):
        agent_type = agent_type.value

    timestamp = int(time.time() * 1000)
    if log_file is None:
        log_file = f"temp_logs/{agent_type}_{timestamp}.log"
    else:
        log_file = f"temp_logs/{log_file}_{agent_type}_{timestamp}.log"

    payload = {"user_prompt": prompt}
    if agent_uuid:
        payload["agent_uuid"] = agent_uuid
    else:
        payload["agent_type"] = agent_type

    pending_tools = None
    agent_uuid_from_stream = None
    
    with open(log_file, "w") as f:
        with httpx.Client(timeout=None) as client:
            with client.stream("POST", f"{BASE_URL}/agent/run", json=payload) as response:
                for line in response.iter_lines():
                    f.write(line)
                    f.write("\n")
                    
                    # Extract agent_uuid from meta_init (HTML entities are escaped)
                    if "meta_init" in line and "agent_uuid" in line:
                        # Unescape HTML entities first, then search
                        unescaped_line = html.unescape(line)
                        match = re.search(r'"agent_uuid":\s*"([^"]+)"', unescaped_line)
                        if match:
                            agent_uuid_from_stream = match.group(1)
                    
                    # Check for awaiting_frontend_tools tag
                    if "awaiting_frontend_tools" in line:
                        # Extract the data attribute
                        match = re.search(r'data="([^"]+)"', line)
                        if match:
                            # Unescape HTML entities
                            data_str = html.unescape(match.group(1))
                            pending_tools = json.loads(data_str)
                            print(f">>> Found awaiting_frontend_tools: {len(pending_tools)} tool(s)")

    return log_file, agent_uuid_from_stream, pending_tools


In [6]:
def submit_tool_results(
    agent_uuid: str,
    tool_results: list[dict],
    log_file: str | None = None,
) -> str:
    """Submit frontend tool results and continue streaming.
    
    Args:
        agent_uuid: The agent UUID to resume
        tool_results: List of {tool_use_id, content, is_error?} dicts
        log_file: Optional log file name prefix
        
    Returns:
        Path to the log file
    """
    timestamp = int(time.time() * 1000)
    if log_file is None:
        log_file = f"temp_logs/tool_results_{timestamp}.log"
    else:
        log_file = f"temp_logs/{log_file}_{timestamp}.log"

    payload = {
        "agent_uuid": agent_uuid,
        "tool_results": tool_results
    }

    with open(log_file, "w") as f:
        with httpx.Client(timeout=None) as client:
            with client.stream("POST", f"{BASE_URL}/agent/tool_results", json=payload) as response:
                for line in response.iter_lines():
                    f.write(line)
                    f.write("\n")

    return log_file


In [7]:
# Step 1: Call agent with a prompt that should trigger user_confirm
# The agent is configured to ask for confirmation when results are over 50

prompt = "Calculate 25 * 4 for me. Confirm with user before proceeding."
print(f"Prompt: {prompt}")
print("-" * 50)

log_path, agent_uuid, pending_tools = call_agent_with_frontend_tools(
    prompt,
    log_file="frontend_tools_step1_pause",
    agent_type=agent_type_enum_v2.agent_frontend_tools
)

print(f"\nLog file: {log_path}")
print(f"Agent UUID: {agent_uuid}")
print(f"Pending tools: {pending_tools}")

FileLink(log_path)


Prompt: Calculate 25 * 4 for me. Confirm with user before proceeding.
--------------------------------------------------
>>> Found awaiting_frontend_tools: 1 tool(s)

Log file: temp_logs/frontend_tools_step1_pause_agent_frontend_tools_1767461323869.log
Agent UUID: d8db26ee-229c-4525-9727-2a8cc8617de5
Pending tools: [{'tool_use_id': 'toolu_018NFVywfpPUKbMSu3Pk8GKm', 'name': 'user_confirm', 'input': {'message': "I'm about to calculate 25 * 4 for you. Should I proceed with this multiplication?"}}]


d:\AI Forge\anthropic-agent\demos\fastapi_server\temp_logs\frontend_tools_step1_pause_agent_frontend_tools_1767461323869.log

In [8]:
# Step 2: Submit frontend tool results (simulating user clicking "yes")
# Only run this if we have pending tools

if pending_tools and agent_uuid:
    # Build tool results - simulate user confirming
    tool_results = [
        {
            "tool_use_id": tool["tool_use_id"],
            "content": "yes",  # User confirmed
            "is_error": False
        }
        for tool in pending_tools
    ]
    
    print(f"Submitting {len(tool_results)} tool result(s):")
    for tr in tool_results:
        print(f"  - {tr['tool_use_id']}: {tr['content']}")
    print("-" * 50)
    
    continuation_log = submit_tool_results(
        agent_uuid=agent_uuid,
        tool_results=tool_results,
        log_file="frontend_tools_step2_continue"
    )
    
    print(f"\nContinuation log: {continuation_log}")
    FileLink(continuation_log)
else:
    print("No pending tools or agent_uuid - agent may have completed without pausing")
    print("Try running the prompt again - Claude's behavior can vary")


Submitting 1 tool result(s):
  - toolu_018NFVywfpPUKbMSu3Pk8GKm: yes
--------------------------------------------------

Continuation log: temp_logs/frontend_tools_step2_continue_1767461330492.log


In [ ]:
# View the full flow in one place
print("=" * 60)
print("FRONTEND TOOLS FLOW SUMMARY")
print("=" * 60)

# Read step 1 log (pause)
if 'log_path' in dir():
    print("\n[STEP 1: Initial Request - Agent Pauses]")
    print("-" * 40)
    with open(log_path, 'r') as f:
        content = f.read()
        # Show last few lines to see the pause marker
        lines = content.strip().split('\n')
        for line in lines[-10:]:
            if line.strip():
                print(line[:200] + "..." if len(line) > 200 else line)

# Read step 2 log (continuation)
if 'continuation_log' in dir() and continuation_log:
    print("\n[STEP 2: Tool Results Submitted - Agent Continues]")
    print("-" * 40)
    with open(continuation_log, 'r') as f:
        content = f.read()
        # Show the content
        lines = content.strip().split('\n')
        for line in lines[:15]:  # First 15 lines of continuation
            if line.strip():
                print(line[:200] + "..." if len(line) > 200 else line)
        if len(lines) > 15:
            print(f"... ({len(lines) - 15} more lines)")
